## 1.1 Import Data and Required Packages

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import recall_score, precision_score, f1_score, fbeta_score, accuracy_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


## 1.2 Set Random Seeds for Reproducibility

In [47]:
np.random.seed(42)
tf.random.set_seed(42)

## 2.1 Import the CSV Data as Pandas DataFrame

In [3]:
customer_df = pd.read_csv('Data/Churn_Modelling.csv')

In [4]:
customer_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 2.2 Drop Unnecessary Columns

In [5]:
customer_df = customer_df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


## 3.1 Prepare X and Y Variables

In [6]:
X = customer_df.drop('Exited', axis=1)
y = customer_df['Exited']


## 3.2 Define Feature Types

In [7]:
categorical_features = ['Geography', 'Gender']
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


## 4.1 Create Column Transformer

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse=False), categorical_features)
    ]
)


## 5.1 Split Data into Training and Test Sets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


## 5.2 Preprocess Data

In [10]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


d:\Projects\customer_churn\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 6.1 Reshape Data for LSTM

In [11]:
X_train = np.expand_dims(X_train, axis=2).astype(np.float32)
X_test = np.expand_dims(X_test, axis=2).astype(np.float32)
y_train_categorical = to_categorical(y_train).astype(np.float32)
y_test_categorical = to_categorical(y_test).astype(np.float32)


## 7.1 Define the Model Building Function (for Keras Tuner)

In [12]:
# def build_model(hp):
#     model = Sequential()
#     lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
#     use_bidirectional = hp.Boolean('use_bidirectional')
    
#     if use_bidirectional:
#         model.add(Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(X_train.shape[1], 1))))
#         model.add(Dropout(0.5))
#         model.add(LSTM(lstm_units // 2))
#     else:
#         model.add(LSTM(lstm_units, input_shape=(X_train.shape[1], 1)))
    
#     model.add(Dropout(0.3))
#     model.add(BatchNormalization())
#     model.add(Dense(y_train_categorical.shape[1], activation='softmax'))
    
#     model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')), 
#                   loss='categorical_crossentropy', 
#                   metrics=['accuracy'])
#     return model
def build_model(hp):
    model = Sequential()
    
    # Set LSTM units, optionally use a predefined value
    lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
    use_bidirectional = hp.Boolean('use_bidirectional')

    # Add Bidirectional LSTM layer
    if use_bidirectional:
        model.add(Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(X_train.shape[1], 1))))
        model.add(Dropout(0.5))  # Dropout after Bidirectional LSTM
        model.add(Bidirectional(LSTM(lstm_units // 2)))  # Add Bidirectional or standard LSTM here based on need
    else:
        model.add(LSTM(lstm_units, return_sequences=False, input_shape=(X_train.shape[1], 1)))
    
    model.add(Dropout(0.3))  # Additional Dropout layer
    model.add(BatchNormalization())  # BatchNormalization layer
    model.add(Dense(y_train_categorical.shape[1], activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model



## 8.1 Initialize Keras Tuner

In [13]:

# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

## 9.1 Define Early Stopping Callback

In [14]:

# Early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

## 9.2 Perform the Hyperparameter Search

In [15]:
tuner.search(X_train, y_train_categorical, epochs=10, validation_split=0.2, callbacks=[stop_early])



Trial 30 Complete [00h 00m 28s]
val_accuracy: 0.8081250190734863

Best val_accuracy So Far: 0.8687499761581421
Total elapsed time: 00h 18m 19s


## 10.1 Retrieve and Print the Best Hyperparameters

In [61]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best LSTM units: {best_hps.get('lstm_units')}")
print(f"Best Bidirectional: {best_hps.get('use_bidirectional')}")
print(f"Best Learning Rate: {best_hps.get('learning_rate')}")


Best LSTM units: 96
Best Bidirectional: True
Best Learning Rate: 0.0071058258296323095


## 10.2 Build the Model with the Best Hyperparameters

In [62]:
best_model = tuner.hypermodel.build(best_hps)


## 11.1 Train the Best Model

In [63]:

# Train the best model
history = best_model.fit(X_train, y_train_categorical, 
                         epochs=10, 
                         batch_size=32, 
                         validation_split=0.2, 
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])

Epoch 1/10
200/200 [==============================] - 13s 27ms/step - loss: 0.5288 - accuracy: 0.7703 - val_loss: 0.4461 - val_accuracy: 0.8000
Epoch 2/10
200/200 [==============================] - 2s 12ms/step - loss: 0.4400 - accuracy: 0.8103 - val_loss: 0.5200 - val_accuracy: 0.8025
Epoch 3/10
200/200 [==============================] - 4s 21ms/step - loss: 0.4391 - accuracy: 0.8091 - val_loss: 0.3810 - val_accuracy: 0.8431
Epoch 4/10
200/200 [==============================] - 4s 18ms/step - loss: 0.4013 - accuracy: 0.8313 - val_loss: 0.3409 - val_accuracy: 0.8569
Epoch 5/10
200/200 [==============================] - 3s 16ms/step - loss: 0.3848 - accuracy: 0.8384 - val_loss: 0.3348 - val_accuracy: 0.8662
Epoch 6/10
200/200 [==============================] - 4s 21ms/step - loss: 0.3809 - accuracy: 0.8378 - val_loss: 0.3350 - val_accuracy: 0.8700
Epoch 7/10
200/200 [==============================] - 3s 16ms/step - loss: 0.3766 - accuracy: 0.8414 - val_loss: 0.3402 - val_accuracy: 0.871

## 12.1 Define the Evaluation Function

In [68]:
# def evaluate_model(model, X_test, y_test_categorical):
#     y_pred = model.predict(X_test)
#     y_pred_classes = np.argmax(y_pred, axis=1)
#     y_true_classes = np.argmax(y_test_categorical, axis=1)
    
#     print("Accuracy Score:", accuracy_score(y_true_classes, y_pred_classes))
#     print("Precision Score:", precision_score(y_true_classes, y_pred_classes, average='macro'))
#     print("Recall Score:", recall_score(y_true_classes, y_pred_classes, average='macro'))
#     print("F1 Score:", f1_score(y_true_classes, y_pred_classes, average='macro'))
#     print("ROC AUC Score:", roc_auc_score(y_test_categorical, y_pred, multi_class='ovr'))

def evaluate_model(model, X_test, y_test_categorical):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test_categorical, axis=1)
    
    print("Accuracy Score:", accuracy_score(y_true_classes, y_pred_classes))
    print("Precision Score:", precision_score(y_true_classes, y_pred_classes, average='weighted'))
    print("Recall Score:", recall_score(y_true_classes, y_pred_classes, average='weighted'))
    print("F1 Score:", f1_score(y_true_classes, y_pred_classes, average='weighted'))
    print("ROC AUC Score:", roc_auc_score(y_test_categorical, y_pred, multi_class='ovr'))



## 13.1 Evaluate the Model

In [69]:
evaluate_model(best_model, X_test, y_test_categorical)


63/63 [==============================] - 0s 6ms/step
Accuracy Score: 0.8575
Precision Score: 0.8503672304258241
Recall Score: 0.8575
F1 Score: 0.8386338220989301
ROC AUC Score: 0.8458890323297105
